# Example Charts

Draco's specification of visualizations (i.e. charts) support single view and multi view  charts. The specification is inspired by [Vega-Lite](https://vega.github.io/vega-lite/).

In [1]:
# Suppressing warnings raised by altair in the background
# (iteration-related deprecation warnings)
import warnings

warnings.filterwarnings("ignore")

In [2]:
import pandas as pd
from vega_datasets import data

# Loading data
weather_data: pd.DataFrame = data.seattle_weather()
# Separating out the positive temperatures for the log scale example
weather_data_positive = weather_data[weather_data["temp_max"] > 0]

In [3]:
from draco import dict_union, dict_to_facts, Draco
from draco.renderer import AltairRenderer
from IPython.display import display
import json

d = Draco()
renderer = AltairRenderer(concat_mode="hconcat")


def show(*args, df: pd.DataFrame = weather_data):
    spec = dict_union(*args)
    prog = dict_to_facts(spec)
    if not d.check_spec(prog):
        print("\n".join(prog))
        print(d.get_violations(prog))
        assert False, "Invalid spec"

    # Display the rendered VL chart and the ASP
    chart = renderer.render(spec, df)
    print(json.dumps(spec, indent=2))
    display(chart)
    display(prog)

## Data for the Examples

As the example, we use a weather dataset.

In [4]:
def data(fields: list[str], df: pd.DataFrame = weather_data) -> dict:
    number_rows, _ = df.shape
    return {
        "number_rows": number_rows,
        "field": [
            x
            for x in [
                {"name": "temp_max", "type": "number", "__id__": "temp_max"},
                {"name": "wind", "type": "number", "__id__": "wind"},
                {"name": "precipitation", "type": "number", "__id__": "precipitation"},
                {"name": "weather", "type": "string", "__id__": "weather"},
            ]
            if x["name"] in fields
        ],
    }

## Single View, Single Mark Charts

### Tick Plot

In [5]:
show(
    data(["temp_max"]),
    {
        "view": [
            {
                "mark": [
                    {
                        "type": "tick",
                        "encoding": [{"channel": "x", "field": "temp_max"}],
                    }
                ],
                "scale": [{"channel": "x", "type": "linear"}],
            }
        ]
    },
)

{
  "number_rows": 1461,
  "field": [
    {
      "name": "temp_max",
      "type": "number",
      "__id__": "temp_max"
    }
  ],
  "view": [
    {
      "mark": [
        {
          "type": "tick",
          "encoding": [
            {
              "channel": "x",
              "field": "temp_max"
            }
          ]
        }
      ],
      "scale": [
        {
          "channel": "x",
          "type": "linear"
        }
      ]
    }
  ]
}


alt.Chart(...)

['attribute(number_rows,root,1461).',
 'entity(field,root,temp_max).',
 'attribute((field,name),temp_max,temp_max).',
 'attribute((field,type),temp_max,number).',
 'entity(view,root,0).',
 'entity(mark,0,1).',
 'attribute((mark,type),1,tick).',
 'entity(encoding,1,2).',
 'attribute((encoding,channel),2,x).',
 'attribute((encoding,field),2,temp_max).',
 'entity(scale,0,3).',
 'attribute((scale,channel),3,x).',
 'attribute((scale,type),3,linear).']

### Tick Plot with a Log Scale

In [6]:
show(
    data(["temp_max"], df=weather_data_positive),
    {
        "view": [
            {
                "mark": [
                    {
                        "type": "tick",
                        "encoding": [{"channel": "x", "field": "temp_max"}],
                    }
                ],
                "scale": [{"channel": "x", "type": "log"}],
            }
        ]
    },
    df=weather_data_positive,
)

{
  "number_rows": 1456,
  "field": [
    {
      "name": "temp_max",
      "type": "number",
      "__id__": "temp_max"
    }
  ],
  "view": [
    {
      "mark": [
        {
          "type": "tick",
          "encoding": [
            {
              "channel": "x",
              "field": "temp_max"
            }
          ]
        }
      ],
      "scale": [
        {
          "channel": "x",
          "type": "log"
        }
      ]
    }
  ]
}


alt.Chart(...)

['attribute(number_rows,root,1456).',
 'entity(field,root,temp_max).',
 'attribute((field,name),temp_max,temp_max).',
 'attribute((field,type),temp_max,number).',
 'entity(view,root,0).',
 'entity(mark,0,1).',
 'attribute((mark,type),1,tick).',
 'entity(encoding,1,2).',
 'attribute((encoding,channel),2,x).',
 'attribute((encoding,field),2,temp_max).',
 'entity(scale,0,3).',
 'attribute((scale,channel),3,x).',
 'attribute((scale,type),3,log).']

### Bar Chart

In [7]:
show(
    data(["weather", "temp_max"]),
    {
        "view": [
            {
                "coordinates": "cartesian",
                "mark": [
                    {
                        "type": "bar",
                        "encoding": [
                            {"channel": "x", "field": "weather"},
                            {
                                "channel": "y",
                                "field": "temp_max",
                                "aggregate": "mean",
                            },
                        ],
                    }
                ],
                "scale": [
                    {"channel": "x", "type": "ordinal"},
                    {"channel": "y", "type": "linear", "zero": "true"},
                ],
            }
        ]
    },
)

{
  "number_rows": 1461,
  "field": [
    {
      "name": "temp_max",
      "type": "number",
      "__id__": "temp_max"
    },
    {
      "name": "weather",
      "type": "string",
      "__id__": "weather"
    }
  ],
  "view": [
    {
      "coordinates": "cartesian",
      "mark": [
        {
          "type": "bar",
          "encoding": [
            {
              "channel": "x",
              "field": "weather"
            },
            {
              "channel": "y",
              "field": "temp_max",
              "aggregate": "mean"
            }
          ]
        }
      ],
      "scale": [
        {
          "channel": "x",
          "type": "ordinal"
        },
        {
          "channel": "y",
          "type": "linear",
          "zero": "true"
        }
      ]
    }
  ]
}


alt.Chart(...)

['attribute(number_rows,root,1461).',
 'entity(field,root,temp_max).',
 'attribute((field,name),temp_max,temp_max).',
 'attribute((field,type),temp_max,number).',
 'entity(field,root,weather).',
 'attribute((field,name),weather,weather).',
 'attribute((field,type),weather,string).',
 'entity(view,root,0).',
 'attribute((view,coordinates),0,cartesian).',
 'entity(mark,0,1).',
 'attribute((mark,type),1,bar).',
 'entity(encoding,1,2).',
 'attribute((encoding,channel),2,x).',
 'attribute((encoding,field),2,weather).',
 'entity(encoding,1,3).',
 'attribute((encoding,channel),3,y).',
 'attribute((encoding,field),3,temp_max).',
 'attribute((encoding,aggregate),3,mean).',
 'entity(scale,0,4).',
 'attribute((scale,channel),4,x).',
 'attribute((scale,type),4,ordinal).',
 'entity(scale,0,5).',
 'attribute((scale,channel),5,y).',
 'attribute((scale,type),5,linear).',
 'attribute((scale,zero),5,true).']

### Radial Chart

In [8]:
show(
    data(["weather", "temp_max"]),
    {
        "view": [
            {
                "coordinates": "polar",
                "mark": [
                    {
                        "type": "bar",
                        "encoding": [
                            {"channel": "x", "field": "weather"},
                            {
                                "channel": "y",
                                "field": "temp_max",
                                "aggregate": "mean",
                            },
                        ],
                    }
                ],
                "scale": [
                    {"channel": "x", "type": "ordinal"},
                    {"channel": "y", "type": "linear", "zero": "true"},
                ],
            }
        ]
    },
)

{
  "number_rows": 1461,
  "field": [
    {
      "name": "temp_max",
      "type": "number",
      "__id__": "temp_max"
    },
    {
      "name": "weather",
      "type": "string",
      "__id__": "weather"
    }
  ],
  "view": [
    {
      "coordinates": "polar",
      "mark": [
        {
          "type": "bar",
          "encoding": [
            {
              "channel": "x",
              "field": "weather"
            },
            {
              "channel": "y",
              "field": "temp_max",
              "aggregate": "mean"
            }
          ]
        }
      ],
      "scale": [
        {
          "channel": "x",
          "type": "ordinal"
        },
        {
          "channel": "y",
          "type": "linear",
          "zero": "true"
        }
      ]
    }
  ]
}


alt.LayerChart(...)

['attribute(number_rows,root,1461).',
 'entity(field,root,temp_max).',
 'attribute((field,name),temp_max,temp_max).',
 'attribute((field,type),temp_max,number).',
 'entity(field,root,weather).',
 'attribute((field,name),weather,weather).',
 'attribute((field,type),weather,string).',
 'entity(view,root,0).',
 'attribute((view,coordinates),0,polar).',
 'entity(mark,0,1).',
 'attribute((mark,type),1,bar).',
 'entity(encoding,1,2).',
 'attribute((encoding,channel),2,x).',
 'attribute((encoding,field),2,weather).',
 'entity(encoding,1,3).',
 'attribute((encoding,channel),3,y).',
 'attribute((encoding,field),3,temp_max).',
 'attribute((encoding,aggregate),3,mean).',
 'entity(scale,0,4).',
 'attribute((scale,channel),4,x).',
 'attribute((scale,type),4,ordinal).',
 'entity(scale,0,5).',
 'attribute((scale,channel),5,y).',
 'attribute((scale,type),5,linear).',
 'attribute((scale,zero),5,true).']

### Histogram

In [9]:
show(
    data(["weather"]),
    {
        "view": [
            {
                "mark": [
                    {
                        "type": "bar",
                        "encoding": [
                            {"channel": "x", "field": "weather"},
                            {"channel": "y", "aggregate": "count"},
                        ],
                    }
                ],
                "scale": [
                    {"channel": "x", "type": "ordinal"},
                    {"channel": "y", "type": "linear", "zero": "true"},
                ],
            }
        ]
    },
)

{
  "number_rows": 1461,
  "field": [
    {
      "name": "weather",
      "type": "string",
      "__id__": "weather"
    }
  ],
  "view": [
    {
      "mark": [
        {
          "type": "bar",
          "encoding": [
            {
              "channel": "x",
              "field": "weather"
            },
            {
              "channel": "y",
              "aggregate": "count"
            }
          ]
        }
      ],
      "scale": [
        {
          "channel": "x",
          "type": "ordinal"
        },
        {
          "channel": "y",
          "type": "linear",
          "zero": "true"
        }
      ]
    }
  ]
}


alt.Chart(...)

['attribute(number_rows,root,1461).',
 'entity(field,root,weather).',
 'attribute((field,name),weather,weather).',
 'attribute((field,type),weather,string).',
 'entity(view,root,0).',
 'entity(mark,0,1).',
 'attribute((mark,type),1,bar).',
 'entity(encoding,1,2).',
 'attribute((encoding,channel),2,x).',
 'attribute((encoding,field),2,weather).',
 'entity(encoding,1,3).',
 'attribute((encoding,channel),3,y).',
 'attribute((encoding,aggregate),3,count).',
 'entity(scale,0,4).',
 'attribute((scale,channel),4,x).',
 'attribute((scale,type),4,ordinal).',
 'entity(scale,0,5).',
 'attribute((scale,channel),5,y).',
 'attribute((scale,type),5,linear).',
 'attribute((scale,zero),5,true).']

### Binned Histogram

In [10]:
show(
    data(["temp_max"]),
    {
        "view": [
            {
                "mark": [
                    {
                        "type": "bar",
                        "encoding": [
                            {"channel": "x", "field": "temp_max", "binning": 10},
                            {"channel": "y", "aggregate": "count"},
                        ],
                    }
                ],
                "scale": [
                    {
                        "channel": "x",
                        "type": "linear",
                    },
                    {"channel": "y", "type": "linear", "zero": "true"},
                ],
            }
        ]
    },
)

{
  "number_rows": 1461,
  "field": [
    {
      "name": "temp_max",
      "type": "number",
      "__id__": "temp_max"
    }
  ],
  "view": [
    {
      "mark": [
        {
          "type": "bar",
          "encoding": [
            {
              "channel": "x",
              "field": "temp_max",
              "binning": 10
            },
            {
              "channel": "y",
              "aggregate": "count"
            }
          ]
        }
      ],
      "scale": [
        {
          "channel": "x",
          "type": "linear"
        },
        {
          "channel": "y",
          "type": "linear",
          "zero": "true"
        }
      ]
    }
  ]
}


alt.Chart(...)

['attribute(number_rows,root,1461).',
 'entity(field,root,temp_max).',
 'attribute((field,name),temp_max,temp_max).',
 'attribute((field,type),temp_max,number).',
 'entity(view,root,0).',
 'entity(mark,0,1).',
 'attribute((mark,type),1,bar).',
 'entity(encoding,1,2).',
 'attribute((encoding,channel),2,x).',
 'attribute((encoding,field),2,temp_max).',
 'attribute((encoding,binning),2,10).',
 'entity(encoding,1,3).',
 'attribute((encoding,channel),3,y).',
 'attribute((encoding,aggregate),3,count).',
 'entity(scale,0,4).',
 'attribute((scale,channel),4,x).',
 'attribute((scale,type),4,linear).',
 'entity(scale,0,5).',
 'attribute((scale,channel),5,y).',
 'attribute((scale,type),5,linear).',
 'attribute((scale,zero),5,true).']

### Scatterplot

In [11]:
show(
    data(["temp_max", "wind"]),
    {
        "view": [
            {
                "mark": [
                    {
                        "type": "point",
                        "encoding": [
                            {"channel": "x", "field": "temp_max"},
                            {"channel": "y", "field": "wind"},
                        ],
                    }
                ],
                "scale": [
                    {"channel": "x", "type": "linear"},
                    {"channel": "y", "type": "linear"},
                ],
            }
        ]
    },
)

{
  "number_rows": 1461,
  "field": [
    {
      "name": "temp_max",
      "type": "number",
      "__id__": "temp_max"
    },
    {
      "name": "wind",
      "type": "number",
      "__id__": "wind"
    }
  ],
  "view": [
    {
      "mark": [
        {
          "type": "point",
          "encoding": [
            {
              "channel": "x",
              "field": "temp_max"
            },
            {
              "channel": "y",
              "field": "wind"
            }
          ]
        }
      ],
      "scale": [
        {
          "channel": "x",
          "type": "linear"
        },
        {
          "channel": "y",
          "type": "linear"
        }
      ]
    }
  ]
}


alt.Chart(...)

['attribute(number_rows,root,1461).',
 'entity(field,root,temp_max).',
 'attribute((field,name),temp_max,temp_max).',
 'attribute((field,type),temp_max,number).',
 'entity(field,root,wind).',
 'attribute((field,name),wind,wind).',
 'attribute((field,type),wind,number).',
 'entity(view,root,0).',
 'entity(mark,0,1).',
 'attribute((mark,type),1,point).',
 'entity(encoding,1,2).',
 'attribute((encoding,channel),2,x).',
 'attribute((encoding,field),2,temp_max).',
 'entity(encoding,1,3).',
 'attribute((encoding,channel),3,y).',
 'attribute((encoding,field),3,wind).',
 'entity(scale,0,4).',
 'attribute((scale,channel),4,x).',
 'attribute((scale,type),4,linear).',
 'entity(scale,0,5).',
 'attribute((scale,channel),5,y).',
 'attribute((scale,type),5,linear).']

### Scatterplot with Color

In [12]:
show(
    data(["temp_max", "wind", "weather"]),
    {
        "view": [
            {
                "mark": [
                    {
                        "type": "point",
                        "encoding": [
                            {"channel": "x", "field": "temp_max"},
                            {"channel": "y", "field": "wind"},
                            {"channel": "color", "field": "weather"},
                        ],
                    }
                ],
                "scale": [
                    {"channel": "x", "type": "linear"},
                    {"channel": "y", "type": "linear"},
                    {"channel": "color", "type": "categorical"},
                ],
            }
        ]
    },
)

{
  "number_rows": 1461,
  "field": [
    {
      "name": "temp_max",
      "type": "number",
      "__id__": "temp_max"
    },
    {
      "name": "wind",
      "type": "number",
      "__id__": "wind"
    },
    {
      "name": "weather",
      "type": "string",
      "__id__": "weather"
    }
  ],
  "view": [
    {
      "mark": [
        {
          "type": "point",
          "encoding": [
            {
              "channel": "x",
              "field": "temp_max"
            },
            {
              "channel": "y",
              "field": "wind"
            },
            {
              "channel": "color",
              "field": "weather"
            }
          ]
        }
      ],
      "scale": [
        {
          "channel": "x",
          "type": "linear"
        },
        {
          "channel": "y",
          "type": "linear"
        },
        {
          "channel": "color",
          "type": "categorical"
        }
      ]
    }
  ]
}


alt.Chart(...)

['attribute(number_rows,root,1461).',
 'entity(field,root,temp_max).',
 'attribute((field,name),temp_max,temp_max).',
 'attribute((field,type),temp_max,number).',
 'entity(field,root,wind).',
 'attribute((field,name),wind,wind).',
 'attribute((field,type),wind,number).',
 'entity(field,root,weather).',
 'attribute((field,name),weather,weather).',
 'attribute((field,type),weather,string).',
 'entity(view,root,0).',
 'entity(mark,0,1).',
 'attribute((mark,type),1,point).',
 'entity(encoding,1,2).',
 'attribute((encoding,channel),2,x).',
 'attribute((encoding,field),2,temp_max).',
 'entity(encoding,1,3).',
 'attribute((encoding,channel),3,y).',
 'attribute((encoding,field),3,wind).',
 'entity(encoding,1,4).',
 'attribute((encoding,channel),4,color).',
 'attribute((encoding,field),4,weather).',
 'entity(scale,0,5).',
 'attribute((scale,channel),5,x).',
 'attribute((scale,type),5,linear).',
 'entity(scale,0,6).',
 'attribute((scale,channel),6,y).',
 'attribute((scale,type),6,linear).',
 'en

### Bubble Chart

In [13]:
show(
    data(["temp_max", "wind", "precipitation"]),
    {
        "view": [
            {
                "mark": [
                    {
                        "type": "point",
                        "encoding": [
                            {"channel": "x", "field": "temp_max"},
                            {"channel": "y", "field": "wind"},
                            {"channel": "size", "field": "precipitation"},
                        ],
                    }
                ],
                "scale": [
                    {"channel": "x", "type": "linear"},
                    {"channel": "y", "type": "linear"},
                    {"channel": "size", "type": "linear"},
                ],
            }
        ]
    },
)

{
  "number_rows": 1461,
  "field": [
    {
      "name": "temp_max",
      "type": "number",
      "__id__": "temp_max"
    },
    {
      "name": "wind",
      "type": "number",
      "__id__": "wind"
    },
    {
      "name": "precipitation",
      "type": "number",
      "__id__": "precipitation"
    }
  ],
  "view": [
    {
      "mark": [
        {
          "type": "point",
          "encoding": [
            {
              "channel": "x",
              "field": "temp_max"
            },
            {
              "channel": "y",
              "field": "wind"
            },
            {
              "channel": "size",
              "field": "precipitation"
            }
          ]
        }
      ],
      "scale": [
        {
          "channel": "x",
          "type": "linear"
        },
        {
          "channel": "y",
          "type": "linear"
        },
        {
          "channel": "size",
          "type": "linear"
        }
      ]
    }
  ]
}


alt.Chart(...)

['attribute(number_rows,root,1461).',
 'entity(field,root,temp_max).',
 'attribute((field,name),temp_max,temp_max).',
 'attribute((field,type),temp_max,number).',
 'entity(field,root,wind).',
 'attribute((field,name),wind,wind).',
 'attribute((field,type),wind,number).',
 'entity(field,root,precipitation).',
 'attribute((field,name),precipitation,precipitation).',
 'attribute((field,type),precipitation,number).',
 'entity(view,root,0).',
 'entity(mark,0,1).',
 'attribute((mark,type),1,point).',
 'entity(encoding,1,2).',
 'attribute((encoding,channel),2,x).',
 'attribute((encoding,field),2,temp_max).',
 'entity(encoding,1,3).',
 'attribute((encoding,channel),3,y).',
 'attribute((encoding,field),3,wind).',
 'entity(encoding,1,4).',
 'attribute((encoding,channel),4,size).',
 'attribute((encoding,field),4,precipitation).',
 'entity(scale,0,5).',
 'attribute((scale,channel),5,x).',
 'attribute((scale,type),5,linear).',
 'entity(scale,0,6).',
 'attribute((scale,channel),6,y).',
 'attribute((

## Stacked Charts

### Pie Chart

In [14]:
show(
    data(["weather", "temp_max"]),
    {
        "view": [
            {
                "coordinates": "polar",
                "mark": [
                    {
                        "type": "bar",
                        "encoding": [
                            {
                                "channel": "y",
                                "aggregate": "count",
                                "stack": "zero",
                            },
                            {"channel": "color", "field": "weather"},
                        ],
                    }
                ],
                "scale": [
                    {"channel": "y", "type": "linear", "zero": "true"},
                    {"channel": "color", "type": "categorical"},
                ],
            }
        ]
    },
)

{
  "number_rows": 1461,
  "field": [
    {
      "name": "temp_max",
      "type": "number",
      "__id__": "temp_max"
    },
    {
      "name": "weather",
      "type": "string",
      "__id__": "weather"
    }
  ],
  "view": [
    {
      "coordinates": "polar",
      "mark": [
        {
          "type": "bar",
          "encoding": [
            {
              "channel": "y",
              "aggregate": "count",
              "stack": "zero"
            },
            {
              "channel": "color",
              "field": "weather"
            }
          ]
        }
      ],
      "scale": [
        {
          "channel": "y",
          "type": "linear",
          "zero": "true"
        },
        {
          "channel": "color",
          "type": "categorical"
        }
      ]
    }
  ]
}


alt.Chart(...)

['attribute(number_rows,root,1461).',
 'entity(field,root,temp_max).',
 'attribute((field,name),temp_max,temp_max).',
 'attribute((field,type),temp_max,number).',
 'entity(field,root,weather).',
 'attribute((field,name),weather,weather).',
 'attribute((field,type),weather,string).',
 'entity(view,root,0).',
 'attribute((view,coordinates),0,polar).',
 'entity(mark,0,1).',
 'attribute((mark,type),1,bar).',
 'entity(encoding,1,2).',
 'attribute((encoding,channel),2,y).',
 'attribute((encoding,aggregate),2,count).',
 'attribute((encoding,stack),2,zero).',
 'entity(encoding,1,3).',
 'attribute((encoding,channel),3,color).',
 'attribute((encoding,field),3,weather).',
 'entity(scale,0,4).',
 'attribute((scale,channel),4,y).',
 'attribute((scale,type),4,linear).',
 'attribute((scale,zero),4,true).',
 'entity(scale,0,5).',
 'attribute((scale,channel),5,color).',
 'attribute((scale,type),5,categorical).']

### Stacked Bar Chart

In [15]:
show(
    data(["temp_max", "weather"]),
    {
        "view": [
            {
                "mark": [
                    {
                        "type": "bar",
                        "encoding": [
                            {"channel": "x", "field": "temp_max", "binning": 10},
                            {"channel": "y", "aggregate": "count", "stack": "zero"},
                            {"channel": "color", "field": "weather"},
                        ],
                    }
                ],
                "scale": [
                    {
                        "channel": "x",
                        "type": "linear",
                    },
                    {"channel": "y", "type": "linear", "zero": "true"},
                    {"channel": "color", "type": "categorical"},
                ],
            }
        ]
    },
)

{
  "number_rows": 1461,
  "field": [
    {
      "name": "temp_max",
      "type": "number",
      "__id__": "temp_max"
    },
    {
      "name": "weather",
      "type": "string",
      "__id__": "weather"
    }
  ],
  "view": [
    {
      "mark": [
        {
          "type": "bar",
          "encoding": [
            {
              "channel": "x",
              "field": "temp_max",
              "binning": 10
            },
            {
              "channel": "y",
              "aggregate": "count",
              "stack": "zero"
            },
            {
              "channel": "color",
              "field": "weather"
            }
          ]
        }
      ],
      "scale": [
        {
          "channel": "x",
          "type": "linear"
        },
        {
          "channel": "y",
          "type": "linear",
          "zero": "true"
        },
        {
          "channel": "color",
          "type": "categorical"
        }
      ]
    }
  ]
}


alt.Chart(...)

['attribute(number_rows,root,1461).',
 'entity(field,root,temp_max).',
 'attribute((field,name),temp_max,temp_max).',
 'attribute((field,type),temp_max,number).',
 'entity(field,root,weather).',
 'attribute((field,name),weather,weather).',
 'attribute((field,type),weather,string).',
 'entity(view,root,0).',
 'entity(mark,0,1).',
 'attribute((mark,type),1,bar).',
 'entity(encoding,1,2).',
 'attribute((encoding,channel),2,x).',
 'attribute((encoding,field),2,temp_max).',
 'attribute((encoding,binning),2,10).',
 'entity(encoding,1,3).',
 'attribute((encoding,channel),3,y).',
 'attribute((encoding,aggregate),3,count).',
 'attribute((encoding,stack),3,zero).',
 'entity(encoding,1,4).',
 'attribute((encoding,channel),4,color).',
 'attribute((encoding,field),4,weather).',
 'entity(scale,0,5).',
 'attribute((scale,channel),5,x).',
 'attribute((scale,type),5,linear).',
 'entity(scale,0,6).',
 'attribute((scale,channel),6,y).',
 'attribute((scale,type),6,linear).',
 'attribute((scale,zero),6,tru

### Normalized (Percentage) Stacked Bar Chart

In [16]:
show(
    data(["temp_max", "weather"]),
    {
        "view": [
            {
                "mark": [
                    {
                        "type": "bar",
                        "encoding": [
                            {
                                "channel": "x",
                                "aggregate": "count",
                                "stack": "normalize",
                            },
                            {"channel": "y", "field": "temp_max", "binning": 10},
                            {"channel": "color", "field": "weather"},
                        ],
                    }
                ],
                "scale": [
                    {"channel": "x", "type": "linear", "zero": "true"},
                    {"channel": "y", "type": "linear"},
                    {"channel": "color", "type": "categorical"},
                ],
            }
        ]
    },
)

{
  "number_rows": 1461,
  "field": [
    {
      "name": "temp_max",
      "type": "number",
      "__id__": "temp_max"
    },
    {
      "name": "weather",
      "type": "string",
      "__id__": "weather"
    }
  ],
  "view": [
    {
      "mark": [
        {
          "type": "bar",
          "encoding": [
            {
              "channel": "x",
              "aggregate": "count",
              "stack": "normalize"
            },
            {
              "channel": "y",
              "field": "temp_max",
              "binning": 10
            },
            {
              "channel": "color",
              "field": "weather"
            }
          ]
        }
      ],
      "scale": [
        {
          "channel": "x",
          "type": "linear",
          "zero": "true"
        },
        {
          "channel": "y",
          "type": "linear"
        },
        {
          "channel": "color",
          "type": "categorical"
        }
      ]
    }
  ]
}


alt.Chart(...)

['attribute(number_rows,root,1461).',
 'entity(field,root,temp_max).',
 'attribute((field,name),temp_max,temp_max).',
 'attribute((field,type),temp_max,number).',
 'entity(field,root,weather).',
 'attribute((field,name),weather,weather).',
 'attribute((field,type),weather,string).',
 'entity(view,root,0).',
 'entity(mark,0,1).',
 'attribute((mark,type),1,bar).',
 'entity(encoding,1,2).',
 'attribute((encoding,channel),2,x).',
 'attribute((encoding,aggregate),2,count).',
 'attribute((encoding,stack),2,normalize).',
 'entity(encoding,1,3).',
 'attribute((encoding,channel),3,y).',
 'attribute((encoding,field),3,temp_max).',
 'attribute((encoding,binning),3,10).',
 'entity(encoding,1,4).',
 'attribute((encoding,channel),4,color).',
 'attribute((encoding,field),4,weather).',
 'entity(scale,0,5).',
 'attribute((scale,channel),5,x).',
 'attribute((scale,type),5,linear).',
 'attribute((scale,zero),5,true).',
 'entity(scale,0,6).',
 'attribute((scale,channel),6,y).',
 'attribute((scale,type),6,

## Multi Mark (Layered) Charts

### Bar with a Tick

In [17]:
show(
    data(["temp_max"]),
    {
        "view": [
            {
                "mark": [
                    {
                        "type": "bar",
                        "encoding": [
                            {
                                "channel": "x",
                                "aggregate": "mean",
                                "field": "temp_max",
                            }
                        ],
                    },
                    {
                        "type": "tick",
                        "encoding": [{"channel": "x", "field": "temp_max"}],
                    },
                ],
                "scale": [{"channel": "x", "type": "linear", "zero": "true"}],
            }
        ]
    },
)

{
  "number_rows": 1461,
  "field": [
    {
      "name": "temp_max",
      "type": "number",
      "__id__": "temp_max"
    }
  ],
  "view": [
    {
      "mark": [
        {
          "type": "bar",
          "encoding": [
            {
              "channel": "x",
              "aggregate": "mean",
              "field": "temp_max"
            }
          ]
        },
        {
          "type": "tick",
          "encoding": [
            {
              "channel": "x",
              "field": "temp_max"
            }
          ]
        }
      ],
      "scale": [
        {
          "channel": "x",
          "type": "linear",
          "zero": "true"
        }
      ]
    }
  ]
}


alt.LayerChart(...)

['attribute(number_rows,root,1461).',
 'entity(field,root,temp_max).',
 'attribute((field,name),temp_max,temp_max).',
 'attribute((field,type),temp_max,number).',
 'entity(view,root,0).',
 'entity(mark,0,1).',
 'attribute((mark,type),1,bar).',
 'entity(encoding,1,2).',
 'attribute((encoding,channel),2,x).',
 'attribute((encoding,aggregate),2,mean).',
 'attribute((encoding,field),2,temp_max).',
 'entity(mark,0,3).',
 'attribute((mark,type),3,tick).',
 'entity(encoding,3,4).',
 'attribute((encoding,channel),4,x).',
 'attribute((encoding,field),4,temp_max).',
 'entity(scale,0,5).',
 'attribute((scale,channel),5,x).',
 'attribute((scale,type),5,linear).',
 'attribute((scale,zero),5,true).']

## Facets

Facet produces horizontal or vertical small multiples.

### Facet Scatterplot into Columns

In [18]:
show(
    data(["temp_max", "wind", "weather"]),
    {
        "view": [
            {
                "mark": [
                    {
                        "type": "point",
                        "encoding": [
                            {"channel": "x", "field": "temp_max"},
                            {"channel": "y", "field": "wind"},
                        ],
                    }
                ],
                "scale": [
                    {"channel": "x", "type": "linear"},
                    {"channel": "y", "type": "linear"},
                ],
                "facet": [
                    {"channel": "col", "field": "weather"},
                ],
            }
        ]
    },
)

{
  "number_rows": 1461,
  "field": [
    {
      "name": "temp_max",
      "type": "number",
      "__id__": "temp_max"
    },
    {
      "name": "wind",
      "type": "number",
      "__id__": "wind"
    },
    {
      "name": "weather",
      "type": "string",
      "__id__": "weather"
    }
  ],
  "view": [
    {
      "mark": [
        {
          "type": "point",
          "encoding": [
            {
              "channel": "x",
              "field": "temp_max"
            },
            {
              "channel": "y",
              "field": "wind"
            }
          ]
        }
      ],
      "scale": [
        {
          "channel": "x",
          "type": "linear"
        },
        {
          "channel": "y",
          "type": "linear"
        }
      ],
      "facet": [
        {
          "channel": "col",
          "field": "weather"
        }
      ]
    }
  ]
}


alt.FacetChart(...)

['attribute(number_rows,root,1461).',
 'entity(field,root,temp_max).',
 'attribute((field,name),temp_max,temp_max).',
 'attribute((field,type),temp_max,number).',
 'entity(field,root,wind).',
 'attribute((field,name),wind,wind).',
 'attribute((field,type),wind,number).',
 'entity(field,root,weather).',
 'attribute((field,name),weather,weather).',
 'attribute((field,type),weather,string).',
 'entity(view,root,0).',
 'entity(mark,0,1).',
 'attribute((mark,type),1,point).',
 'entity(encoding,1,2).',
 'attribute((encoding,channel),2,x).',
 'attribute((encoding,field),2,temp_max).',
 'entity(encoding,1,3).',
 'attribute((encoding,channel),3,y).',
 'attribute((encoding,field),3,wind).',
 'entity(scale,0,4).',
 'attribute((scale,channel),4,x).',
 'attribute((scale,type),4,linear).',
 'entity(scale,0,5).',
 'attribute((scale,channel),5,y).',
 'attribute((scale,type),5,linear).',
 'entity(facet,0,6).',
 'attribute((facet,channel),6,col).',
 'attribute((facet,field),6,weather).']

### Facet Scatterplot by Binned Data into Columns

In [19]:
show(
    data(["temp_max", "wind", "weather"]),
    {
        "view": [
            {
                "mark": [
                    {
                        "type": "point",
                        "encoding": [
                            {"channel": "x", "field": "weather"},
                            {"channel": "y", "field": "wind"},
                        ],
                    }
                ],
                "scale": [
                    {"channel": "x", "type": "ordinal"},
                    {"channel": "y", "type": "linear"},
                ],
                "facet": [
                    {"channel": "col", "field": "temp_max", "binning": 10},
                ],
            }
        ]
    },
)

{
  "number_rows": 1461,
  "field": [
    {
      "name": "temp_max",
      "type": "number",
      "__id__": "temp_max"
    },
    {
      "name": "wind",
      "type": "number",
      "__id__": "wind"
    },
    {
      "name": "weather",
      "type": "string",
      "__id__": "weather"
    }
  ],
  "view": [
    {
      "mark": [
        {
          "type": "point",
          "encoding": [
            {
              "channel": "x",
              "field": "weather"
            },
            {
              "channel": "y",
              "field": "wind"
            }
          ]
        }
      ],
      "scale": [
        {
          "channel": "x",
          "type": "ordinal"
        },
        {
          "channel": "y",
          "type": "linear"
        }
      ],
      "facet": [
        {
          "channel": "col",
          "field": "temp_max",
          "binning": 10
        }
      ]
    }
  ]
}


alt.FacetChart(...)

['attribute(number_rows,root,1461).',
 'entity(field,root,temp_max).',
 'attribute((field,name),temp_max,temp_max).',
 'attribute((field,type),temp_max,number).',
 'entity(field,root,wind).',
 'attribute((field,name),wind,wind).',
 'attribute((field,type),wind,number).',
 'entity(field,root,weather).',
 'attribute((field,name),weather,weather).',
 'attribute((field,type),weather,string).',
 'entity(view,root,0).',
 'entity(mark,0,1).',
 'attribute((mark,type),1,point).',
 'entity(encoding,1,2).',
 'attribute((encoding,channel),2,x).',
 'attribute((encoding,field),2,weather).',
 'entity(encoding,1,3).',
 'attribute((encoding,channel),3,y).',
 'attribute((encoding,field),3,wind).',
 'entity(scale,0,4).',
 'attribute((scale,channel),4,x).',
 'attribute((scale,type),4,ordinal).',
 'entity(scale,0,5).',
 'attribute((scale,channel),5,y).',
 'attribute((scale,type),5,linear).',
 'entity(facet,0,6).',
 'attribute((facet,channel),6,col).',
 'attribute((facet,field),6,temp_max).',
 'attribute((f

## Multiple Views

Any of the specifications above are a single view. You can also create multiple views.

### Tick Plot and Histogram

In [20]:
show(
    data(["temp_max", "weather"]),
    {
        "view": [
            {
                "mark": [
                    {
                        "type": "tick",
                        "encoding": [{"channel": "y", "field": "temp_max"}],
                    }
                ],
                "scale": [{"channel": "y", "type": "linear", "zero": "true"}],
            },
            {
                "mark": [
                    {
                        "type": "bar",
                        "encoding": [
                            {"channel": "x", "field": "weather"},
                            {"channel": "y", "aggregate": "count"},
                        ],
                    }
                ],
                "scale": [
                    {"channel": "x", "type": "ordinal"},
                    {"channel": "y", "type": "linear", "zero": "true"},
                ],
            },
        ]
    },
)

{
  "number_rows": 1461,
  "field": [
    {
      "name": "temp_max",
      "type": "number",
      "__id__": "temp_max"
    },
    {
      "name": "weather",
      "type": "string",
      "__id__": "weather"
    }
  ],
  "view": [
    {
      "mark": [
        {
          "type": "tick",
          "encoding": [
            {
              "channel": "y",
              "field": "temp_max"
            }
          ]
        }
      ],
      "scale": [
        {
          "channel": "y",
          "type": "linear",
          "zero": "true"
        }
      ]
    },
    {
      "mark": [
        {
          "type": "bar",
          "encoding": [
            {
              "channel": "x",
              "field": "weather"
            },
            {
              "channel": "y",
              "aggregate": "count"
            }
          ]
        }
      ],
      "scale": [
        {
          "channel": "x",
          "type": "ordinal"
        },
        {
          "channel": "y",
       

alt.HConcatChart(...)

['attribute(number_rows,root,1461).',
 'entity(field,root,temp_max).',
 'attribute((field,name),temp_max,temp_max).',
 'attribute((field,type),temp_max,number).',
 'entity(field,root,weather).',
 'attribute((field,name),weather,weather).',
 'attribute((field,type),weather,string).',
 'entity(view,root,0).',
 'entity(mark,0,1).',
 'attribute((mark,type),1,tick).',
 'entity(encoding,1,2).',
 'attribute((encoding,channel),2,y).',
 'attribute((encoding,field),2,temp_max).',
 'entity(scale,0,3).',
 'attribute((scale,channel),3,y).',
 'attribute((scale,type),3,linear).',
 'attribute((scale,zero),3,true).',
 'entity(view,root,4).',
 'entity(mark,4,5).',
 'attribute((mark,type),5,bar).',
 'entity(encoding,5,6).',
 'attribute((encoding,channel),6,x).',
 'attribute((encoding,field),6,weather).',
 'entity(encoding,5,7).',
 'attribute((encoding,channel),7,y).',
 'attribute((encoding,aggregate),7,count).',
 'entity(scale,4,8).',
 'attribute((scale,channel),8,x).',
 'attribute((scale,type),8,ordinal

### Tick Plot and Histogram with Shared Y-Scale

In [21]:
show(
    data(["temp_max", "weather"]),
    {
        "view": [
            {
                "mark": [
                    {
                        "type": "tick",
                        "encoding": [{"channel": "y", "field": "temp_max"}],
                    }
                ],
            },
            {
                "mark": [
                    {
                        "type": "bar",
                        "encoding": [
                            {
                                "channel": "y",
                                "field": "temp_max",
                                "aggregate": "mean",
                            },
                            {"channel": "x", "field": "weather"},
                        ],
                    }
                ],
                "scale": [{"channel": "x", "type": "ordinal"}],
            },
        ],
        "scale": [{"channel": "y", "type": "linear", "zero": "true"}],
    },
)

{
  "number_rows": 1461,
  "field": [
    {
      "name": "temp_max",
      "type": "number",
      "__id__": "temp_max"
    },
    {
      "name": "weather",
      "type": "string",
      "__id__": "weather"
    }
  ],
  "view": [
    {
      "mark": [
        {
          "type": "tick",
          "encoding": [
            {
              "channel": "y",
              "field": "temp_max"
            }
          ]
        }
      ]
    },
    {
      "mark": [
        {
          "type": "bar",
          "encoding": [
            {
              "channel": "y",
              "field": "temp_max",
              "aggregate": "mean"
            },
            {
              "channel": "x",
              "field": "weather"
            }
          ]
        }
      ],
      "scale": [
        {
          "channel": "x",
          "type": "ordinal"
        }
      ]
    }
  ],
  "scale": [
    {
      "channel": "y",
      "type": "linear",
      "zero": "true"
    }
  ]
}


alt.HConcatChart(...)

['attribute(number_rows,root,1461).',
 'entity(field,root,temp_max).',
 'attribute((field,name),temp_max,temp_max).',
 'attribute((field,type),temp_max,number).',
 'entity(field,root,weather).',
 'attribute((field,name),weather,weather).',
 'attribute((field,type),weather,string).',
 'entity(view,root,0).',
 'entity(mark,0,1).',
 'attribute((mark,type),1,tick).',
 'entity(encoding,1,2).',
 'attribute((encoding,channel),2,y).',
 'attribute((encoding,field),2,temp_max).',
 'entity(view,root,3).',
 'entity(mark,3,4).',
 'attribute((mark,type),4,bar).',
 'entity(encoding,4,5).',
 'attribute((encoding,channel),5,y).',
 'attribute((encoding,field),5,temp_max).',
 'attribute((encoding,aggregate),5,mean).',
 'entity(encoding,4,6).',
 'attribute((encoding,channel),6,x).',
 'attribute((encoding,field),6,weather).',
 'entity(scale,3,7).',
 'attribute((scale,channel),7,x).',
 'attribute((scale,type),7,ordinal).',
 'entity(scale,root,8).',
 'attribute((scale,channel),8,y).',
 'attribute((scale,type